In [10]:
import calendar
import datetime
import importlib 
import numpy as np
import os

from parcels import Field, FieldSet, ParticleSet,Variable, JITParticle

In [2]:
import OP_functions_shared as OP
import PBDEs_OP_Kernels_shared as PBDE
import Susans_Model_Driver as Susan

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [11]:
importlib.reload(PBDE)
importlib.reload(OP)
importlib.reload(Susan)

<module 'Susans_Model_Driver' from '/ocean/sallen/allen/research/Meopar/ANALYSIS/analysis-vicente/Ocean_Parcels/SHARED/Susans_Model_Driver.py'>

In [ ]:
year, month = 2021, 1
sim_length = 500 # days
number_outputs = 100
Susan.PBDEs_OP_run(year, month, sim_length, number_outputs)

2976


In [6]:
year, month = 2021, 1
sim_length = 200 # days

(start_time, data_length, duration, delta_t, 
 release_particles_every, number_particles, output_interval) = Susan.timings(year, month, sim_length, 100)


2976


In [12]:
field_set, constants = Susan.set_fieldsets_and_constants(start_time, data_length, delta_t)

2.1990716561059998e-06 0.0001851680394672206 9.258830603076085e-05
3.968246094721017e-06 9.259216392498715e-06 1.5854896284750453e-09
0.0002768034297052155 0.004428854875283448
2.44140625e-05 0.006510416666666667


In [ ]:
path = {'out': '/home/sallen/MEOPAR/ANALYSIS/analysis-vicente/Ocean_Parcels/SHARED/',}
fn = 'particles_out.zarr'
outfile_states = os.path.join(path['out'], fn)

### Particle Class ###

In [ ]:
class MPParticle(JITParticle):
    # Status Variable #
    status = Variable('status', initial=(np.random.rand(number_particles) > constants['fraction_colloidal']).astype(int) - 2)
    vvl_factor = Variable('fact', initial=1)
    release_time = Variable('release_time', 
                            initial=np.arange(0, release_particles_every*number_particles, release_particles_every))

In [ ]:
pset_states = ParticleSet(field_set, pclass=MPParticle, lon=constants['Iona_clon']*np.ones(number_particles), 
                          depth=constants['Iona_z']*np.ones(number_particles), lat = constants['Iona_clat']*np.ones(number_particles))

In [ ]:
output_interval = datetime.timedelta(seconds=sim_length*84600/100)
output_file = pset_states.ParticleFile(name=outfile_states, outputdt=output_interval)
KE = (pset_states.Kernel(PBDE.PBDEs_states) + pset_states.Kernel(PBDE.Sinking) 
      + pset_states.Kernel(PBDE.Advection)
      + pset_states.Kernel(PBDE.turb_mix) + pset_states.Kernel(PBDE.resuspension)
     )
# Currently have ab/de-sorption, sinking and vertical mixing and resuspension for 5 particles for a bit over an hour.

In [ ]:
#KE = PBDE.Sinking #+ pset.Kernel(tu_mix) #if you want to add more kernels use:  KE = tu_mix + NAME_KERNEL1 + ...
pset_states.execute(KE, runtime=duration, dt=delta_t, output_file=output_file,
                    )

In [18]:
kappa = 0.42
zo, rho = 0.07, 1024                              # from SalishSeaCast
log_z_star = np.log(zo)
cdmin, cdmax = 0.0075, 2                          # from SalishSeaCast
lowere3t_o2 = zo * np.exp(kappa / np.sqrt(cdmax))
uppere3t_o2 = zo * np.exp(kappa / np.sqrt(cdmin))
print (log_z_star, lowere3t_o2, uppere3t_o2)

tau_crit = 0.05
tau_bury_crit = 0.8
tau_constant = tau_crit / ((kappa ** 2) * rho)
tau_constant_lower = tau_crit / (rho * cdmax)
tau_constant_upper = tau_crit / (rho * cdmin)
print (tau_constant, tau_constant_lower, tau_constant_upper)
#field_set.add_constant('tau_bury_constant', tau_bury_crit / ((kappa ** 2) * rho))
#field_set.add_constant('tau_bury_constant_lower', tau_bury_crit / (rho * cdmax))
#field_set.add_constant('tau_bury_constant_upper', tau_bury_crit / (rho * cdmin))


-2.659260036932778 0.09420564355590687 8.939522930296924
0.0002768034297052155 2.44140625e-05 0.006510416666666667


In [30]:
import math
# Say e3t = 0.2, then:
e3t_val_o2 = 0.1
log_e3t = math.log(e3t_val_o2)
np.sqrt((log_e3t - log_z_star) **2 * tau_constant)

0.00593414747409176

In [25]:
np.sqrt(0.004530306207594626)

0.06730754941011169